# Decision trees (DTs): imputer dataset

Decision trees using sklearn for the database Imputer
Imputer include features that are categorical

In [28]:
#USEFUL! To get the number lies (Ctrl+Shift+P)
#Look for cell line numbers and press Enter

Download Database 

In [62]:
# Libraries
import pandas as pd
import subprocess
import seaborn.apionly as sns
from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import numpy as np


# 1 Database
raw_data = sns.load_dataset('titanic')
predictors = ['pclass','sex','age','sibsp','parch','fare','embarked','alone','adult_male']

raw_data.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


*** Rápido ejemplo de dummies!***

In [30]:
#Database: Ejemplo usando dummies
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'],'C': [1, 2, 3]})
print(df)

#Crear base de datos con dummies: lo que hace es tomar las variables que son categoricas y las divide por
#columnas dependiendo de los valores que tiene, conservando la fila en que se encuantran, por ejemplo:
#A tiene a y b: se divide en dos columnas
#B tiene a,b,c: se divide en tres columnas

print('\n \n Base con dummies:')
pd.get_dummies(df, prefix=['col1', 'col2'])

   A  B  C
0  a  b  1
1  b  a  2
2  a  c  3

 
 Base con dummies:


,C,col1_a,col1_b,col2_a,col2_b,col2_c
0,1,1.0,0.0,0.0,1.0,0.0
1,2,0.0,1.0,1.0,0.0,0.0
2,3,1.0,0.0,0.0,0.0,1.0


### Continuemos... Ahora a obtener los tipos de variables y convertir categoricas en dummies

Crear el vector de features con los nuevos dummies

In [31]:
# 2 Variables
# Get categorical variables
categorical = ['sex','embarked']
# Get numeric variables
numeric = [c for c in predictors if c not in categorical]
# Get target
target='survived'

# 3 Convert categorical variables t
#Convert categorical variables into dummy/indicator variables
encoded_data = pd.get_dummies(raw_data[predictors], columns=categorical)
encoded_data.head()

,pclass,age,sibsp,parch,fare,alone,adult_male,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,3,22.0,1,0,7.2500,False,True,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,False,False,1.0,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,True,False,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,False,False,1.0,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,True,True,0.0,1.0,0.0,0.0,1.0


Separar descriptive features y target featires.
Convertir a float32

In [32]:
#Data and target
imputer = Imputer()
X = imputer.fit_transform(encoded_data).astype('float32')
Y = raw_data[target].astype('float32')

### Entrenar árbol

In [33]:
#Train the tree
model = DecisionTreeClassifier(min_samples_leaf=10, max_depth=3)
model.fit(X, Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [34]:
#Export to pdf
export_graphviz(model,
                out_file='tree.dot',
                feature_names=encoded_data.columns,
                proportion=True,
                filled=True,
                impurity=False)

subprocess.call(['dot', '-Tpdf', 'tree.dot', '-o' 'tree.pdf'])

#Show pdf created
from IPython.display import IFrame
IFrame("./tree.pdf",width="500",height='300')

# Decision trees: iris dataset

DecisionTreeClassifier is capable of both binary (where the labels are [-1, 1]) classification and multiclass (where the labels are [0, …, K-1]) classification.

Decision trees using sklearn for the database Iris with multiclass classification
Multiclass classification: classifying into one of three or more classes (not binary)

In [9]:
# Libraries
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz

#Import dataset
iris = load_iris()
feature_names_iris=iris.feature_names
print('Targets:  ',list(iris.target_names))
print('Features: ',list(iris.feature_names))
#iris.data[1]

Targets:   ['setosa', 'versicolor', 'virginica']
Features:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [10]:
iris.data[1:5]

array([[ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

Entrenar árbol

In [11]:
#Train the tree
model_clf = DecisionTreeClassifier()   #Other arguments: (min_samples_leaf=10, max_depth=3)
model_clf.fit(iris.data , iris.target) #Data and target


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [12]:
#Export to pdf
export_graphviz(model_clf,
                out_file='tree_iris.dot',
                feature_names=feature_names_iris,
                proportion=True,
                filled=True,
                impurity=False)

subprocess.call(['dot', '-Tpdf', 'tree_iris.dot', '-o' 'tree_iris.pdf'])

0

In [13]:
#Show pdf created
from IPython.display import IFrame
IFrame("./tree_iris.pdf",width="500",height='300')

In [14]:
#Test tree with and input:
#['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
x=[0.                ,0.                , 3.                ,1.66]

#Predict class
class_iris=model_clf.predict([x])
print('Classified as: ',iris.target_names[class_iris[0]],'\n')

#Target names
print('Targets: ', list(iris.target_names),':')

#Probablity of each class 
prob_iris=model_clf.predict_proba([x])
print('Porcentages: ', prob_iris[0]*100)

Classified as:  virginica 

Targets:  ['setosa', 'versicolor', 'virginica'] :
Porcentages:  [   0.    0.  100.]


# Decision tree using Entropy: iris dataset  

In [149]:
iris.data


array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

In [154]:
type(iris.target)

numpy.ndarray

In [15]:
# Libraries
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz

#Import dataset
iris = load_iris()
feature_names_iris=iris.feature_names

#Train the tree
model_clf_entropy = DecisionTreeClassifier(criterion='gini')   #Other arguments: (min_samples_leaf=10, max_depth=3)
model_clf_entropy.fit(iris.data , iris.target) #Data and target

#Export to pdf
export_graphviz(model_clf_entropy,
                out_file='tree_iris_entropy.dot',
                feature_names=feature_names_iris,
                proportion=True,
                filled=True,
                impurity=False)

subprocess.call(['dot', '-Tpdf', 'tree_iris_entropy.dot', '-o' 'tree_iris_entropy.pdf'])


#Show pdf created
from IPython.display import IFrame
IFrame("./tree_iris_entropy.pdf",width="500",height='300')

In [27]:
# Libraries
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz

#Import dataset
iris = load_iris()
feature_names_iris=iris.feature_names

#Train the tree
model_clf_entropy = DecisionTreeClassifier(criterion='entropy')   #Other arguments: (min_samples_leaf=10, max_depth=3)
model_clf_entropy.fit(iris.data , iris.target) #Data and target

#Export to pdf
export_graphviz(model_clf_entropy,
                out_file='tree_iris_entropy.dot',
                feature_names=feature_names_iris,
                proportion=True,
                filled=True,
                impurity=False)

subprocess.call(['dot', '-Tpdf', 'tree_iris_entropy.dot', '-o' 'tree_iris_entropy.pdf'])


#Show pdf created
from IPython.display import IFrame
IFrame("./tree_iris_entropy.pdf",width="500",height='300')